# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f68108588e0>
├── 'a' --> tensor([[-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922]])
└── 'x' --> <FastTreeValue 0x7f68108cbfd0>
    └── 'c' --> tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                        [ 2.5440, -0.8773, -0.1070,  0.5311],
                        [ 1.4577,  0.4764,  0.3511, -0.2365]])

In [4]:
t.a

tensor([[-0.4506,  1.6953,  0.0186],
        [-0.8136, -0.2265,  0.2922]])

In [5]:
%timeit t.a

59.1 ns ± 0.0873 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f68108588e0>
├── 'a' --> tensor([[-1.7519, -0.0439,  1.3940],
│                   [ 0.5891,  0.4093, -0.7613]])
└── 'x' --> <FastTreeValue 0x7f68108cbfd0>
    └── 'c' --> tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                        [ 2.5440, -0.8773, -0.1070,  0.5311],
                        [ 1.4577,  0.4764,  0.3511, -0.2365]])

In [7]:
%timeit t.a = new_value

63.4 ns ± 0.0629 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4506,  1.6953,  0.0186],
               [-0.8136, -0.2265,  0.2922]]),
    x: Batch(
           c: tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365]]),
       ),
)

In [10]:
b.a

tensor([[-0.4506,  1.6953,  0.0186],
        [-0.8136, -0.2265,  0.2922]])

In [11]:
%timeit b.a

54.9 ns ± 0.0213 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.4963,  0.6181,  0.2560],
               [ 1.1632, -1.0302, -0.3039]]),
    x: Batch(
           c: tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 2.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f681085f7c0>
├── 'a' --> tensor([[[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]],
│           
│                   [[-0.4506,  1.6953,  0.0186],
│                    [-0.8136, -0.2265,  0.2922]]])
└── 'x' --> <FastTreeValue 0x7f676d1eb220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 42.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f676650f8b0>
├── 'a' --> tensor([[-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922],
│                   [-0.4506,  1.6953,  0.0186],
│                   [-0.8136, -0.2265,  0.2922]])
└── 'x' --> <FastTreeValue 0x7f676d154b50>
    └── 'c' --> tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                        [ 2.5440, -0.8773, -0.1070,  0.5311],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 44 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.1 µs ± 32.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0764, -0.2585,  0.3614, -0.5254],
                       [ 2.5440, -0.8773, -0.1070,  0.5311],
                       [ 1.4577,  0.4764,  0.3511, -0.2365]],
              
                      [[ 1.0764, -0.2585,  0.3614, -0.5254],
                       [ 2.5440, -0.8773, -0.1070,  0.5311],
                       [ 1.4577,  0.4764,  0.3511, -0.2365]],
              
                      [[ 1.0764, -0.2585,  0.3614, -0.5254],
                       [ 2.5440, -0.8773, -0.1070,  0.5311],
                       [ 1.4577,  0.4764,  0.3511, -0.2365]],
              
                      [[ 1.0764, -0.2585,  0.3614, -0.5254],
                       [ 2.5440, -0.8773, -0.1070,  0.5311],
                       [ 1.4577,  0.4764,  0.3511, -0.2365]],
              
                      [[ 1.0764, -0.2585,  0.3614, -0.5254],
                       [ 2.5440, -0.8773, -0.1070,  0.5311],
                       [ 1.4577,  0.4764,  0.3511, -0.2365]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365],
                      [ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365],
                      [ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365],
                      [ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365],
                      [ 1.0764, -0.2585,  0.3614, -0.5254],
                      [ 2.5440, -0.8773, -0.1070,  0.5311],
                      [ 1.4577,  0.4764,  0.3511, -0.2365],
                      [ 1.0764, -0.2585,  0.3614, -0.5254],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 4.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
